# Automatic Hyperparameter Tuning / AutoML

> Also known as AutoML, Automatic Machine Learning, Meta-optimization, Meta-learning, and so on.

So not only you want to build neural networks and other machine learning algorithms, but also you want to find the best hyperparameters for them automatically. We'll here demonstrate how it's possible with Neuraxle. 

## AutoML loop

Neuraxle provides an AutoML step which can perform automatic tuning of the hyperparameters of any neuraxle-defined pipeline. It also provides multiple hyperparameter research strategies, interfaces to store and load results and a high degree of customisability. The following tutorial will provide an example of the main features of the AutoML step.

### Define your problem
For this tutorial, we'll use a synthetic binary classification dataset generated by sklearn with 10k samples.

In [ ]:
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
import shutil

n_classes = 2
classes = list(range(n_classes))

def generate_classification_data():
    data_inputs, expected_outputs = make_classification(
        n_samples=10000,
        n_informative=10,
        n_repeated=0,
        n_classes=n_classes,
        n_features=15,
        n_clusters_per_class=1,
        class_sep=1.0,
        flip_y=0,
        weights=[0.5, 0.5]
    )

    X_train, X_test, y_train, y_test = train_test_split(
        data_inputs,
        expected_outputs,
        test_size=0.20
    )
    
    return X_train, y_train, X_test, y_test


X_train, y_train, X_test, y_test = generate_classification_data()

### Define your pipeline

Consider a sklearn classifier with a defined hyperparameter space.

In [ ]:
from neuraxle.base import BaseTransformer
from neuraxle.pipeline import Pipeline
from neuraxle.hyperparams.space import HyperparameterSpace
from neuraxle.steps.numpy import NumpyRavel
from neuraxle.steps.output_handlers import OutputTransformerWrapper
from typing import List

# sklearn classifiers, and sklearn wrapper for neuraxle 
from neuraxle.steps.sklearn import SKLearnWrapper
from sklearn.tree import DecisionTreeClassifier, ExtraTreeClassifier
from sklearn.linear_model import RidgeClassifier, LogisticRegression, SGDClassifier


# neuraxle distributions 
from neuraxle.hyperparams.distributions import Choice, RandInt, Boolean, LogUniform, FixedHyperparameter
    
classifier = SKLearnWrapper(SGDClassifier(), 
                   HyperparameterSpace({
                       'max_iter': FixedHyperparameter(1),
                       'learning_rate': FixedHyperparameter('constant'),
                       'loss': FixedHyperparameter('hinge'),#Choice(['hinge','log', 'modified_huber', 'squared_hinge']),
                       'penalty':Choice(['l2', 'l1', 'elasticnet']),
                       'alpha':LogUniform(1e-9, 1e-3),
                       'eta0': LogUniform(1e-6, 1e-3)
                   }),
                   use_partial_fit=True,
                   partial_fit_kwargs={'classes': classes})


Consider a pipeline which performs the following: shuffles the data on training and process incrementally the data in batch of 32 samples with the classifier.

In [ ]:
from neuraxle.steps.flow import ChooseOneStepOf, TrainOnlyWrapper
from neuraxle.steps.data import DataShuffler
from neuraxle.pipeline import MiniBatchSequentialPipeline

p: Pipeline = Pipeline([
    TrainOnlyWrapper(DataShuffler()),
    MiniBatchSequentialPipeline([
        classifier
    ], batch_size=32,
    keep_incomplete_batch=True)
])

### Choose a validation splitter

Let's pick the [ValidationSplitter](https://www.neuraxle.org/stable/api/metaopt/neuraxle.metaopt.auto_ml.html?highlight=validationsplitter#neuraxle.metaopt.auto_ml.ValidationSplitter) which is enough for most machine learning problems:

In [ ]:
from neuraxle.metaopt.validation import ValidationSplitter

validation_splitter = ValidationSplitter(validation_size=0.20)

This validation splitter will perform a single 80%-20% split of the dataset.

We have also implemented [KFoldCrossValidationSplitter](https://www.neuraxle.org/stable/api/metaopt/neuraxle.metaopt.auto_ml.html?highlight=kfoldcross#neuraxle.metaopt.auto_ml.KFoldCrossValidationSplitter) if you are interested in using cross-validation for your machine learning problem.

### Define a the main scoring metric with a first [MetricsCallback](https://www.neuraxle.org/stable/api/metaopt/neuraxle.metaopt.callbacks.html?highlight=metriccallback#neuraxle.metaopt.callbacks.MetricCallback)

Let's pick the accuracy score as our scoring metric for now, as the dataset does not suffer from data imbalance: 

In [10]:
from neuraxle.metaopt.callbacks import MetricCallback
from sklearn.metrics import accuracy_score

scoring_callback = MetricCallback(
    'accuracy',
    metric_function=accuracy_score,
    higher_score_is_better=True,
    log_metrics=True
)

### Add other metric callbacks with [MetricCallback](https://www.neuraxle.org/stable/api/metaopt/neuraxle.metaopt.callbacks.html?highlight=metriccallback#neuraxle.metaopt.callbacks.MetricCallback) (optional)

During training, we'll monitor the f1 score, the precision and the recall on the training and validation dataset. We will make use of the sklearn implementation of these metric, using a weighted average since we are in a multiclass setting.

In [12]:
from neuraxle.metaopt.callbacks import MetricCallback
from sklearn.metrics import f1_score, precision_score, recall_score

callbacks = [
    scoring_callback,
    MetricCallback('f1', metric_function=f1_score, higher_score_is_better=True, log_metrics=False),
    MetricCallback('precision', metric_function=precision_score, higher_score_is_better=True, log_metrics=False),
    MetricCallback('recall', metric_function=recall_score, higher_score_is_better=True, log_metrics=False)
]

Multiple callbacks, such as EarlyStoppingCallback and SaveBestModelCallback are available as part of Neuraxle. You could also implement your own callback by inheriting from [BaseCallback](https://www.neuraxle.org/stable/api/metaopt/neuraxle.metaopt.callbacks.html?highlight=basecallback#neuraxle.metaopt.callbacks.BaseCallback).

### Select an hyperparams repository 

For the sake of this example, let's pick the [VanillaHyperparamsRepository](https://www.neuraxle.org/stable/api/metaopt/data/neuraxle.metaopt.data.repositories.repo.html#neuraxle.metaopt.repositories.repo.VanillaHyperparamsRepository) because we don't need to persist trial files. [HyperparamsOnDiskRepository](https://www.neuraxle.org/stable/api/metaopt/data/neuraxle.metaopt.repositories.json.html#neuraxle.metaopt.repositories.json.HyperparamsOnDiskRepository)​ is also available if you want to use json files to persist trials.


In [14]:
from neuraxle.metaopt.repositories.repo import VanillaHyperparamsRepository
from neuraxle.base import ExecutionContext

hyperparams_repository = VanillaHyperparamsRepository(cache_folder=ExecutionContext.get_new_cache_folder())

### Select an hyperparams optimizer 

We'll use the Tree Parzen Estimator algorithm as a research strategy.

In [20]:
from neuraxle.metaopt.hyperopt.tpe import TreeParzenEstimator

hyperparams_optimizer = TreeParzenEstimator(number_of_initial_random_step=5)

Note: as of now, [TreeParzenEstimator](https://www.neuraxle.org/stable/api/metaopt/hyperopt/neuraxle.metaopt.hyperopt.tpe.html?highlight=tpe#neuraxle.metaopt.hyperopt.tpe.TreeParzenEstimator) is the first usable Apache 2 public TPE implementation ! Thanks to Éric Hamel for implementing this amazing algorithm. 

Also note that if you don't define the hyperparams_optimizer argument, [RandomSearchSampler](https://www.neuraxle.org/stable/api/metaopt/neuraxle.metaopt.validation.html#neuraxle.metaopt.validation.RandomSearchSampler) will be used by default.

### Create, and launch AutoML loop

We are now ready to create the AutoML step that will execute the training loop:

In [22]:
from neuraxle.metaopt.auto_ml import AutoML
    
n_trials = 10
n_epochs = 10
    
auto_ml = AutoML(
    pipeline=p,
    validation_splitter=validation_splitter,
    n_trials=n_trials,
    epochs=n_epochs,
    callbacks=callbacks,
    hyperparams_repository=hyperparams_repository,
    refit_best_trial=True
)

#### Run 10 trials 

In [23]:
auto_ml = auto_ml.fit(X_train, y_train)

[2022-04-19_03:27:50.%f][INFO    ][neuraxle.default_project.default_client.0.AutoML][base.py:950]: Planned!


/usr/local/lib/python3.7/dist-packages/neuraxle/metaopt/validation.py:204: UserWarning: Warning: changed GridExplorationSampler.expected_n_trials from 0 to 12 due to high amount of trials. This may lead to a non-reproducible search using RandomSearch as a fallback past this point.
  f"Warning: changed {self.__class__.__name__}.expected_n_trials from "
[2022-04-19_03:27:50.%f][INFO    ][neuraxle.default_project.default_client.0.AutoML][base.py:950]: Planned!


[2022-04-19_03:27:50.%f][INFO    ][neuraxle.default_project.default_client.0.AutoML][base.py:950]: Hyperparameters: OrderedDict([   (   'MiniBatchSequentialPipeline__SKLearnWrapper_SGDClassifier__alpha',
                    1e-09),
                (   'MiniBatchSequentialPipeline__SKLearnWrapper_SGDClassifier__eta0',
                    1e-06),
                (   'MiniBatchSequentialPipeline__SKLearnWrapper_SGDClassifier__learning_rate',
                    'constant'),
                (   'MiniBatchSequentialPipeline__SKLearnWrapper_SGDClassifier__loss',
                    'hinge'),
                (   'MiniBatchSequentialPipeline__SKLearnWrapper_SGDClassifier__max_iter',
                    1),
                (   'MiniBatchSequentialPipeline__SKLearnWrapper_SGDClassifier__penalty',
                    'elasticnet')])


[2022-04-19_03:27:50.%f][INFO    ][neuraxle.default_project.default_client.0.AutoML][base.py:950]: Hyperparameters: OrderedDict([   (   'MiniBatchSequentialPipeline__SKLearnWrapper_SGDClassifier__alpha',
                    1e-09),
                (   'MiniBatchSequentialPipeline__SKLearnWrapper_SGDClassifier__eta0',
                    1e-06),
                (   'MiniBatchSequentialPipeline__SKLearnWrapper_SGDClassifier__learning_rate',
                    'constant'),
                (   'MiniBatchSequentialPipeline__SKLearnWrapper_SGDClassifier__loss',
                    'hinge'),
                (   'MiniBatchSequentialPipeline__SKLearnWrapper_SGDClassifier__max_iter',
                    1),
                (   'MiniBatchSequentialPipeline__SKLearnWrapper_SGDClassifier__penalty',
                    'elasticnet')])


[2022-04-19_03:27:50.%f][INFO    ][neuraxle.default_project.default_client.0.AutoML][base.py:950]: Status: TrialStatus.PLANNED


...

[2022-04-19_03:28:32.%f][INFO    ][neuraxle.default_project.default_client.0.AutoML][base.py:950]: Finished round hp search!


[2022-04-19_03:28:32.%f][INFO    ][neuraxle.default_project.default_client.0.AutoML][base.py:950]: Best hyperparameters found for metric 'accuracy':


[2022-04-19_03:28:32.%f][INFO    ][neuraxle.default_project.default_client.0.AutoML][base.py:950]: Best hyperparameters found for metric 'accuracy':


[2022-04-19_03:28:32.%f][INFO    ][neuraxle.default_project.default_client.0.AutoML][base.py:950]: Hyperparameters: OrderedDict([   (   'MiniBatchSequentialPipeline__SKLearnWrapper_SGDClassifier__alpha',
                    0.001),
                (   'MiniBatchSequentialPipeline__SKLearnWrapper_SGDClassifier__eta0',
                    0.001),
                (   'MiniBatchSequentialPipeline__SKLearnWrapper_SGDClassifier__learning_rate',
                    'constant'),
                (   'MiniBatchSequentialPipeline__SKLearnWrapper_SGDClassifier__loss',
                    'hinge'),
                (   'MiniBatchSequentialPipeline__SKLearnWrapper_SGDClassifier__max_iter',
                    1),
                (   'MiniBatchSequentialPipeline__SKLearnWrapper_SGDClassifier__penalty',
                    'l1')])


[2022-04-19_03:28:32.%f][INFO    ][neuraxle.default_project.default_client.0.AutoML][base.py:950]: Hyperparameters: OrderedDict([   (   'MiniBatchSequentialPipeline__SKLearnWrapper_SGDClassifier__alpha',
                    0.001),
                (   'MiniBatchSequentialPipeline__SKLearnWrapper_SGDClassifier__eta0',
                    0.001),
                (   'MiniBatchSequentialPipeline__SKLearnWrapper_SGDClassifier__learning_rate',
                    'constant'),
                (   'MiniBatchSequentialPipeline__SKLearnWrapper_SGDClassifier__loss',
                    'hinge'),
                (   'MiniBatchSequentialPipeline__SKLearnWrapper_SGDClassifier__max_iter',
                    1),
                (   'MiniBatchSequentialPipeline__SKLearnWrapper_SGDClassifier__penalty',
                    'l1')])


### Get best model and measure test accuracy

In [24]:
y_pred = auto_ml.predict(X_test)

accuracy = accuracy_score(y_true=y_test, y_pred=y_pred)
print("Test accuracy score:", accuracy)


Test accuracy score: 0.9775


### Additional note : model selection as an hyperparameter

It is possible to consider the choice of a model as a hyperparameter of its own and define an AutoML pipeline with this in mind. Consider multiple sklearn classifiers that each have their own hyperparms: 

In [25]:
classifiers: List[BaseTransformer] = [
    SKLearnWrapper(DecisionTreeClassifier(), HyperparameterSpace({
            'criterion': Choice(['gini', 'entropy']),
            'splitter': Choice(['best', 'random']),
            'min_samples_leaf': RandInt(2, 5),
            'min_samples_split': RandInt(2, 4)
        })).set_name('DecisionTreeClassifier'),
    Pipeline([
            OutputTransformerWrapper(NumpyRavel()),
            SKLearnWrapper(RidgeClassifier(), HyperparameterSpace({
                'alpha': Choice([(0.0, 1.0, 10.0), (0.0, 10.0, 100.0)]),
                'fit_intercept': Boolean(),
                'normalize': Boolean()
    }))
    ]).set_name('RidgeClassifier'),
    Pipeline([
        OutputTransformerWrapper(NumpyRavel()),
        SKLearnWrapper(LogisticRegression(), HyperparameterSpace({
            'C': LogUniform(0.01, 10.0),
            'fit_intercept': Boolean(),
            'dual': Boolean(),
            'penalty': Choice(['l1', 'l2']),
            'max_iter': RandInt(20, 200)
        }))
    ]).set_name('LogisticRegression')
]

To have an hyperparameter which decides which model to use, we recommend usage of the ChooseOneStepOf abstraction.

In [26]:
from neuraxle.steps.flow import ChooseOneStepOf

p: Pipeline = Pipeline([
    ChooseOneStepOf(classifiers)
])

If provided to an AutoML loop, a classifier will be sampled at beginning of every trial.